# Assignment
## UserInterface
#### This Notebook allows the user to use an interface to control a robot in its environment and to see some results with some graph.

## Mode Cell
#### This cell allow the user to choose the mode he want to use for the robot. This is done using three button each of which calls up the service "/mode" that allows the set of the mode.

In [1]:
import rospy
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist
from mode_srv.srv import Mode
from first_mode_srv.srv import Firstmode
from controller_srv.srv import Controller
from nav_msgs.msg import Odometry
import ipywidgets as widgets
from IPython.display import display
button1 = widgets.Button(description="Mode 1")
button2 = widgets.Button(description="Mode 2")
button3 = widgets.Button(description="Mode 3")
output = widgets.Output()



display(button1)
display(button2)
display(button3)
display(output)

def on_button_clicked1(b):    
    with output:
        rospy.wait_for_service("/mode")
        client1 = rospy.ServiceProxy("/mode", Mode)
        client1(1, 1)
        

def on_button_clicked2(b):
    with output:
        rospy.wait_for_service("/mode")
        client1 = rospy.ServiceProxy("/mode", Mode)
        client1(1, 2)
         

def on_button_clicked3(b):
    with output:
        rospy.wait_for_service("/mode")
        client1 = rospy.ServiceProxy("/mode", Mode)
        client1(1, 3)
        
        
button1.on_click(on_button_clicked1)
button2.on_click(on_button_clicked2)
button3.on_click(on_button_clicked3)



Button(description='Mode 1', style=ButtonStyle())

Button(description='Mode 2', style=ButtonStyle())

Button(description='Mode 3', style=ButtonStyle())

Output()

## First Mode
#### The user can use this cell to send the goals that the robot have to reach for the first mode. This is done using two FloatText widgets that allow the user to insert the values x and y, than there is a button that, through his callback, call the service "/first_mode" that set the goal chosen by the user. 

In [2]:
import ipywidgets as widgets

x = widgets.FloatText()
y = widgets.FloatText()
button8 = widgets.Button(description="Send")
output = widgets.Output()

display(x)
display(y)
display(button8)
display(output)
def on_button_clicked8(b):    
    with output:
        rospy.wait_for_service("/first_mode")
        client2 = rospy.ServiceProxy("/first_mode", Firstmode)
        client2(1, x.value, y.value)
        
button8.on_click(on_button_clicked8) 

FloatText(value=0.0)

FloatText(value=0.0)

Button(description='Send', style=ButtonStyle())

Output()

#### Part of the first mode is the count of the reached goals. In this cell there is a button that the user can press when he want to see the graph of reached goals against total sends goals. This is possible because in the callback of the button is called the "/reach_goal"  service that give back the values of total sends goals and reached goals, than these values are used to make the graph.

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import rospy
import tf
from nav_msgs.msg import Odometry
import ipywidgets as widgets
from reach_srv.srv import Reach

button9 = widgets.Button(description="plot")
output = widgets.Output()

display(button9)
display(output)
def on_button_clicked9(b):    
    with output:
        rospy.wait_for_service("/reach_goal")
        client2 = rospy.ServiceProxy("/reach_goal", Reach)
        resp=client2(2, 0, 0)
        
        tot_goals=resp.goals
        reached_goals=resp.reached
        index = np.arange(1);
        eConf = {'ecolor': '0.3'}
        plt.bar(index, tot_goals, align='edge', width=0.35, alpha=0.40,
        color='b', error_kw=eConf, label='Tot Goals')
        plt.bar(index+0.35, reached_goals, align='edge', width=0.35,
        alpha=0.40, color='r', error_kw=eConf, label='Reached goals')
        plt.xlabel('Goals')
        plt.ylabel('Values')
        plt.title('Number of reached point on total sends goals')
        plt.xticks(index + 0.35, (''))
        plt.legend()
        plt.show()

button9.on_click(on_button_clicked9) 




Button(description='plot', style=ButtonStyle())

Output()

## Second and Third Mode
#### This cell allow the user to control directly the movements of the robot choosing the direction that the robot have to take. This is done in the same way used for the choose of the mode. There are four button which call the "/controller" service that set in the core script witch is the right command. 

In [4]:
button4 = widgets.Button(description="Go on")
button5 = widgets.Button(description="Stop")
button6 = widgets.Button(description="Right")
button7 = widgets.Button(description="Left")
output = widgets.Output()



display(button4)
display(button5)
display(button6)
display(button7)
display(output)
#handle changes, the observe method of the widget can be used to register a callback.
def on_button_clicked4(b):    
    with output:
        rospy.wait_for_service("/controller")
        client3 = rospy.ServiceProxy("/controller", Controller)
        client3(1, 1)
        

def on_button_clicked5(b):
    with output:
        rospy.wait_for_service("/controller")
        client3 = rospy.ServiceProxy("/controller", Controller)
        client3(1, 2)
         

def on_button_clicked6(b):
    with output:
        rospy.wait_for_service("/controller")
        client3 = rospy.ServiceProxy("/controller", Controller)
        client3(1, 3)
        
def on_button_clicked7(b):
    with output:
        rospy.wait_for_service("/controller")
        client3 = rospy.ServiceProxy("/controller", Controller)
        client3(1, 4)
        
button4.on_click(on_button_clicked4)
button5.on_click(on_button_clicked5)
button6.on_click(on_button_clicked6)
button7.on_click(on_button_clicked7)

Button(description='Go on', style=ButtonStyle())

Button(description='Stop', style=ButtonStyle())

Button(description='Right', style=ButtonStyle())

Button(description='Left', style=ButtonStyle())

Output()

## Other Graph
#### The first graph allow the user to see the position of the robot in real time. This is possible because there is the call to the topic "/odom" that give back the coordinates of the robot in the environment, than thanks to the FuncAnimation function, these are plotted.

In [1]:
import matplotlib.pyplot as plt
import rospy
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation

%matplotlib widget
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = self.ax.plot([], [], 'ro')
        self.x_data, self.y_data = [], []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    
rospy.init_node('odom_visualizer_node')
vis = Visualiser()

sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)    
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### The second graph allow the user to see a 3D map, thanks to the ros3d functions, that show to the user the laser scan of the robot and the map in which the robot is moving.

In [1]:
from jupyros import ros3d
import os
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090")
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client)
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc,
tf_client=tf_client)
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client,
path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]
v

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…